In [1]:
import numpy as np
import os
import nrrd
import glob
import math
#import vtk
vtkpaths= glob.glob(os.path.expanduser("~")+'/src/negtrainsamples/*.save')
datapath = os.path.expanduser("~")+'/DATA/Case  '
print(vtkpaths)



['/home/ubuntu/src/negtrainsamples/case070vtk-neg.save', '/home/ubuntu/src/negtrainsamples/case077vtk-neg.save', '/home/ubuntu/src/negtrainsamples/case066vtk-neg.save', '/home/ubuntu/src/negtrainsamples/case054vtk-neg.save', '/home/ubuntu/src/negtrainsamples/case033vtk-neg.save', '/home/ubuntu/src/negtrainsamples/case064vtk-neg.save', '/home/ubuntu/src/negtrainsamples/case050vtk-neg.save']


In [2]:
def print_full(x):
    np.set_printoptions(threshold=np.inf)
    print(x)
    np.set_printoptions(threshold=1000)
    
def str2float(list):
    newlist=[]
    n=list.ndim
    if n>1:       
        for j in range(len(list)):
            temp=[float(i) for i in list[j]]
            newlist.append(temp)
    else:
        newlist=[float(i) for i in list]
    return newlist    

def gettransmat(spacing,origin):
    spacing.append([0.,0.,0.])
    origin.append(1.)
    spacing=np.array(spacing)
    origin=np.array(origin)
    transmat=np.linalg.inv(np.c_[ spacing,origin ])
    return transmat

def transcoord(data,transmat):
    data=data.transpose()
    data=np.append(data,np.array([[1. for i in range(5000)]]),axis=0)
#    print(data,data.shape)
    
    newdata=np.dot(transmat,data)
    newdata=np.rint(newdata)
    newdata=newdata.astype(int)
#    print(newdata,newdata.shape)
    
    return newdata       

def getintensity(imgneedles):
    meangray=[]
    intensity=[]
    rng=[-1,0,1]
    for i in range(len(imgneedles)):
        mean=0
        graylevel=[]
        for j in range(5000):
            temp=2000
            for x1 in rng:
                for x2 in rng:
                    if nrrddata[0][imgneedles[i][0][j]+x1][imgneedles[i][1][j]+x2][imgneedles[i][2][j]]<temp:
                        temp=nrrddata[0][imgneedles[i][0][j]+x1][imgneedles[i][1][j]+x2][imgneedles[i][2][j]]
            gray=temp
            graylevel.append(gray)
            mean+=gray
        mean/=5000
        meangray.append(mean)
        intensity.append(graylevel)
    return np.array(meangray),np.array(intensity)

def gethistogram(intensity):
    totalhis=[]
    for i in range(len(intensity)):
        upbound=10
        histogram=[]
        while upbound<105:
            num=sum(j < upbound and j>=(upbound-10) for j in intensity[i])/5000
            histogram.append(num)
            upbound+=10
        num=1-sum(histogram)
        histogram.append(num)
        totalhis.append(histogram)
        totalhis2=np.around(totalhis,2)
    return totalhis2         

def getsegmean(intensity):
    totalsegmean=[]
    for i in range(len(intensity)):
        upbound=500
        segmean=[]
        while upbound<=5000:
            num=np.mean(intensity[i][upbound-500:upbound])
            segmean.append(num)
            upbound+=500
        totalsegmean.append(segmean)
        totalsegmean2=np.around(totalsegmean,2)
    return totalsegmean2        

def sortseq(needles):
    newseq=[]
    for i in range(len(needles)):
        temp=needles[i][needles[i][:,2].argsort()]
        #print(temp)
        newseq.append(temp)
    return newseq    

totalmean=[]
totalhisto=[]
totalintensity=[]
count=0
count2=0
for vtkpath in vtkpaths:
    needles = np.load(vtkpath)
    print(vtkpath)
    needles[:,:,:2]=needles[:,:,:2]*(-1.)
    needles=sortseq(needles)
    needles=np.array(needles)
    print(needles.shape) 
    
    if int(vtkpath[-15:-12])==21:
        continue
        
    dirpath=datapath+vtkpath[-15:-12]
    nrrdfiles=glob.glob(dirpath+'/NRRD/Manual_CY/*.nrrd')
    if not nrrdfiles:
        continue
    for nrrdfile in nrrdfiles:
        if not 'label' in nrrdfile:
            nrrdpath=nrrdfile
            break 
    print(nrrdpath)
    count+=1
    nrrddata = nrrd.read(nrrdpath)
    print(nrrddata[1])
    print(nrrddata[1]['space origin'])  
    nrrdspacing=str2float(np.array(nrrddata[1]['space directions']))
    nrrdsporig=str2float(np.array(nrrddata[1]['space origin'])) 
    nrrdsize=str2float(np.array(nrrddata[1]['sizes'])) 

    transmat=gettransmat(nrrdspacing,nrrdsporig)

    imgneedles=[]
    for i in range(len(needles)):
        temp=transcoord(needles[i],transmat)
        imgneedles.append(temp)
    imgneedles=np.array(imgneedles)
    #print(imgneedles)
    index1=[] 
    for i,x in enumerate(imgneedles):
        if (np.amax(x[0])>nrrdsize[0]-2 or np.amax(x[1])>nrrdsize[1]-2 or np.amax(x[2])>nrrdsize[2]-2 or np.amin(x[1])<0 or np.amin(x[2])<0 or np.amin(x[0])<0):
            index1.append(i)
    
    print(index1)
    index1=np.array(index1)
    imgneedles=np.delete(imgneedles,index1,0)

    mean,intensity=getintensity(imgneedles)
    histo=gethistogram(intensity)
    segmean=getsegmean(intensity)
    
    if count==1:
        totalmean=mean
        totalhisto=histo
        totalintensity=intensity
        totalsegmean=segmean
    else:
        totalmean=np.concatenate((totalmean, mean), axis=0)
        totalhisto=np.concatenate((totalhisto, histo), axis=0)
        totalintensity=np.concatenate((totalintensity, intensity), axis=0)
        totalsegmean=np.concatenate((totalsegmean, segmean), axis=0)

#     f = open('mean'+vtkpath[-11:-8]+'.save', 'wb')
#     np.save(f, mean)
#     f.close
#     f = open('intensity'+vtkpath[-11:-8]+'.save', 'wb')
#     np.save(f, intensity)
#     f.close
#     f = open('histo'+vtkpath[-11:-8]+'.save', 'wb')
#     np.save(f, histo)
#     f.close
#     if int(vtkpath[-11:-8])==64 or int(vtkpath[-11:-8])==33 or int(vtkpath[-11:-8])==76:
#         count2+=1
#         if count2==1:
#             testmean=mean
#             testhisto=histo
#             testintensity=intensity
#             testsegmean=segmean
#         else:
#             testmean=np.concatenate((testmean, mean), axis=0)
#             testhisto=np.concatenate((testhisto, histo), axis=0)
#             testintensity=np.concatenate((testintensity, intensity), axis=0)
#             testsegmean=np.concatenate((testsegmean, segmean), axis=0)
#     else:
#         if count==1:
#             trainmean=mean
#             trainhisto=histo
#             trainintensity=intensity
#             trainsegmean=segmean
#         else:
#             trainmean=np.concatenate((trainmean, mean), axis=0)
#             trainhisto=np.concatenate((trainhisto, histo), axis=0)
#             trainintensity=np.concatenate((trainintensity, intensity), axis=0)
#             trainsegmean=np.concatenate((trainsegmean, segmean), axis=0)
    print(mean)
    print(intensity,intensity.shape)
    print(histo)
    print(segmean)

# meangray=[]   
# for i in range(len(imgneedles)):
#     num=0       
#     for j in range(5000):
#         if nrrddata[0][imgneedles[i][0][j]][imgneedles[i][1][j]][imgneedles[i][2][j]]>100:
#             num+=1
#     meangray.append(num)



# meangray=[]   
# for j in range(5000):
#     print(nrrddata[0][imgneedles[0][0][j]][imgneedles[0][1][j]][imgneedles[0][2][j]])

    
    
#meanfilter = [mean for mean in meangray if mean<70]
# meanfilter = [i for i,x in enumerate(mean) if x<60 and histo[i][0]>0.2]
# print(meanfilter,len(meanfilter))
            

/home/ubuntu/src/negtrainsamples/case070vtk-neg.save
(208, 5000, 3)
/home/ubuntu/DATA/Case  070/NRRD/Manual_CY/14 AX space trial 1.6 SLICE COVER TEMPLATE.nrrd
{'dimension': 3, 'space': 'left-posterior-superior', 'space origin': ['-148.06295776367187', '-90.153846740722656', '-145.76899719238281'], 'type': 'unsigned short', 'keyvaluepairs': {}, 'space directions': [['0.46875', '0', '0'], ['0', '0.46875', '0'], ['0', '0', '1.600006103515625']], 'sizes': [640, 416, 144], 'kinds': ['domain', 'domain', 'domain'], 'encoding': 'gzip', 'endian': 'little'}
['-148.06295776367187', '-90.153846740722656', '-145.76899719238281']
[11, 19, 31, 52, 58, 69, 80, 106, 116, 131, 141, 143, 145, 152, 174, 179, 180, 197]
[  30.3624   29.3564   28.8448   26.4428   39.3924   42.158    34.168
   71.912    31.3444   47.7892   30.9012   17.9056   61.2524   33.4324
   28.2616   27.8976   21.7104   43.4276   82.886    19.506    46.9672
   22.4844   31.7104   26.2076   46.9672   78.3632   31.7176   30.4384
   42.965

In [6]:
totalmin=np.amin(totalintensity,axis=1)
totalmax=np.amax(totalintensity,axis=1)
totalvar=np.var(totalintensity,axis=1)
totalmed=np.median(totalintensity,axis=1)

print(totalmin.shape)
print(totalmax.shape)
print(totalvar.shape)
print(totalmed.shape)  


(879,)
(879,)
(879,)
(879,)


In [7]:
f = open('negtotalhisto.save', 'wb')
np.save(f, totalhisto)
f.close
f = open('negtotalmean.save', 'wb')
np.save(f, totalmean)
f.close
f = open('negtotalintensity.save', 'wb')
np.save(f, totalintensity)
f.close
f = open('negtotalsegmean.save', 'wb')
np.save(f, totalsegmean)
f.close
f = open('negtotalmin.save', 'wb')
np.save(f, totalmin)
f.close
f = open('negtotalvar.save', 'wb')
np.save(f, totalvar)
f.close
f = open('negtotalmed.save', 'wb')
np.save(f, totalmed)
f.close
f = open('negtotalmax.save', 'wb')
np.save(f, totalmax)
f.close
print('ok')


ok


In [8]:
b=np.mean(totalhisto,axis=0)
print(b)

[ 0.15682594  0.1040273   0.12147895  0.08634812  0.05624573  0.04712173
  0.04135381  0.03790671  0.03387941  0.02940842  0.2849033 ]
